This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5).

In this notebook, we identify curves of genus 7 with an irrational $g^2_6$ which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 1 hour for completion.

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of the quadratic twist $X$ of $\mathbb{P}^2 \times \mathbb{P}^2$. These can be identified with the $\mathbb{F}_4$-points of $\mathbb{P}^2$.

In [2]:
F = GF(2)
P.<x0,x1,x2,y0,y1,y2> = PolynomialRing(F, 6)

In [3]:
F4.<a> = GF(4)
S14 = [vector(F4,t) for t in itertools.product(F4, repeat=3) if not all(i==0 for i in t) and [i for i in t if i != 0][0] == 1]
S = [(x, vector(F4, tuple(i.conjugate() for i in x))) for x in S14]
for v in S:
    for x in v:
        x.set_immutable()
len(S)

21

In [4]:
F16.<a2> = F4.extension(2)
S4 = list(itertools.product(S14, repeat=2))
for v in S4:
    for x in v:
        x.set_immutable()
len(S4)

441

In [5]:
F64.<a3> = F4.extension(3)
S164 = [vector(F64,t) for t in itertools.product(F64, repeat=3) if not all(i==0 for i in t) and [i for i in t if i != 0][0] == 1]
S8 = [(x, vector(F64, tuple(i.conjugate() for i in x))) for x in S164]
for v in S8:
    for x in v:
        x.set_immutable()
len(S8)

4161

Construct the group $\mathrm{GL}(3,\mathbb{F}_4)$, presented as a subgroup of $\mathrm{GL}(6, \mathbb{F}_4)$.

In [6]:
l0 = [block_matrix(2,2,[g.matrix(), 0, 0, g.matrix().conjugate()], subdivide=False) for g in GL(3,F4).gens()]
G0 = GL(6,F4).subgroup(l0)
G0.order()

181440

Use an orbit lookup tree to find $\mathrm{GL}(3,\mathbb{F}_4)$-orbit representatives for 7-tuples of $\mathbb{F}_2$-points in $X$.

In [7]:
def apply_group_elem(g, x, S=S, a=a):
    g1 = g.submatrix(nrows=3,ncols=3)
    g2 = g.submatrix(row=3,col=3)
    v1 = g1*x[0]
    v2 = g2*x[1]
    for i in range(3):
        if (v1, v2) in S:
            v1.set_immutable()
            v2.set_immutable()
            return (v1, v2)
        v1 = a*v1
        v2 = (a+1)*v2
    raise ValueError

In [8]:
amat = diagonal_matrix(F4, [a,a,a,a+1,a+1,a+1])

In [9]:
def stabilizer(x):
    G1 = vec_stab(Matrix(F4, x[0]), transpose=True)
    l0 = [block_matrix(2,2,[g.matrix(),0,0,g.matrix().conjugate()], subdivide=False) for g in G1.gens()]
    return GL(6, F4).subgroup(l0)

In [10]:
def optimized_rep(g):
    return g.matrix()

In [11]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep}
tree = build_orbit_tree(G0, S, 7, methods, verbose=False)

In [12]:
l6 = green_nodes(tree, 6)
len(l6)

8

In [13]:
l7 = green_nodes(tree, 7)
len(l7)

10

For each orbit representative, for each 2-dimensional space of $(1,1)$-hypersurfaces passing through those points, solve for $(2,2)$-hypersurfaces passing through exactly those $\mathbb{F}_2$-rational points, and impose the desired point counts over $\mathbb{F}_4$ and $\mathbb{F}_8$.

In [14]:
monos11 = [x0*y0, x1*y1, x2*y2,
           x1*y0 + x0*y1, a*x1*y0 + (a+1)*x0*y1,
           x2*y0 + x0*y2, a*x2*y0 + (a+1)*x0*y2,
           x1*y2 + x2*y1, a*x1*y2 + (a+1)*x2*y1]
monos2a = [prod(i) for i in itertools.product([prod(j) for j in itertools.combinations_with_replacement([x0,x1,x2],2)], 
                                              [prod(j) for j in itertools.combinations_with_replacement([y0,y1,y2],2)])]
monos2b = [prod(t) for t in itertools.product(monos11, repeat=2)]
tmp = [vector(F4, (x.coefficient(mu.exponents()[0]) for mu in monos2a)) for x in monos2b]
tmp2 = []
monos2 = []
for i in range(len(monos2b)):
    if Matrix(F4, tmp2 + [tmp[i]]).rank() > Matrix(F4, tmp2).rank():
        tmp2.append(tmp[i])
        monos2.append(monos2b[i])
len(monos2)

36

In [15]:
coords1 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos11)) for x in S}
coords2 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos2)) for x in S}

In [16]:
def redundancy(gens, F=F, F4=F4, monos11=monos11, monos2a=monos2a, tmp2=tmp2):
    ans = []
    for m in monos11:
        for v in gens:
            tmp3 = vector(F4, ((v*m).coefficient(mu.exponents()[0]) for mu in monos2a))
            w = Matrix(tmp2 + [tmp3]).left_kernel().gens()[0][:-1]
            ans.append(vector(F, (w[i] for i in range(len(w)))))
    return ans

In [17]:
l2a = []
l2b = []
for vecs in l6+l7:
#    print(len(l2a), len(l2b))
    V = Matrix([coords1[x] for x in vecs]).right_kernel()
    for W in V.subspaces(2):
        (v1, v2) = W.basis()
        pts = [x for x in S if coords1[x]*v1 == 0 and coords1[x]*v2 == 0]
        perp = Matrix([coords2[x] for x in pts])
        target = vector(F, (0 if x in vecs else 1 for x in pts))
        gens1 = sum(v1[i]*monos11[i] for i in range(9))
        gens2 = sum(v2[i]*monos11[i] for i in range(9))
        pts2 = 0
        pts3 = 0
        for w in solve_right_iterator(perp, target, redundancy, [gens1, gens2]):
            gens3 = sum(w[i]*monos2[i] for i in range(36))
            if pts2 == 0:
                pts2 = [x for x in S4 if x not in S and gens1(*x[0], *x[1]) == 0 and gens2(*x[0], *x[1]) == 0]
            s = sum(1 for x in pts2 if gens3(*x[0], *x[1]) == 0)
            if (vecs in l6 and s != 12) or (vecs in l7 and s != 8):
                continue
            if pts3 == 0:
                pts3 = [x for x in S8 if x not in S and gens1(*x[0], *x[1]) == 0 and gens2(*x[0], *x[1]) == 0]
            s = sum(1 for x in pts3 if gens3(*x[0], *x[1]) == 0)
            if (vecs in l6 and s != 6) or (vecs in l7 and s != 0):
                continue
            if vecs in l6:
                l2a.append((v1, v2, w))
            elif vecs in l7:
                l2b.append((v1, v2, w))

In [18]:
len(l2a), len(l2b)

(886, 134)

Impose the desired point count conditions over $\mathbb{F}_{2^i}$ for $i=4,5$, yielding no results.

In [19]:
def vecs_to_gens(vecs):
    return [sum(vecs[0][i]*monos11[i] for i in range(9)),
            sum(vecs[1][i]*monos11[i] for i in range(9)),
            sum(vecs[2][i]*monos2[i] for i in range(36))]

In [20]:
count_gens = {}
for n in range(1, 6):
    if n%2 == 1:
        count_gens[n] = [x0^(2^n)+y0, x1^(2^n)+y1, x2^(2^n)+y2, y0^(2^n)+x0, y1^(2^n)+x1, y2^(2^n)+x2]
    else:
        count_gens[n] = [x^(2^n)+x for x in P.gens()]

def count_by_ideal(vecs, n):
    J = P.change_ring(F4).ideal(vecs_to_gens(vecs) + count_gens[n])
    if n%2 == 1:
        return (J.vector_space_dimension() - 1) // (2^(2*n)-1)
    else:
        return (J.vector_space_dimension() - 2^(3*n+1) + 1) // (2^n-1)^2

In [21]:
l4a = [vecs for vecs in l2a if count_by_ideal(vecs, 4) == 18]
len(l4a)

1

In [22]:
l4b = [vecs for vecs in l2b if count_by_ideal(vecs, 4) == 31]
len(l4b)

12

In [23]:
l5a = [vecs for vecs in l4a if count_by_ideal(vecs, 5) == 6]
assert not l5a

In [24]:
l5b = [vecs for vecs in l4b if count_by_ideal(vecs, 5) in [12, 22]]
assert not l5b

Finish up.

In [ ]:
closeout([], genus=7)